# PC2_LP2_UNALM

Integrantes:
    - Ana
    - Stefano
    - Elian

- ¿Tiene API?
 No tiene API

- Crear un repositorio de trabajo en GitHub y trabajar de manera colaborativa con los integrantes de su grupo.
https://github.com/analisset7/PC2_LP2_UNALM

- Descargar la información solicitada y estructurarla de acuerdo al formato en excel: "CHGenesOrdenadosConVecinos.xlsx".



In [12]:
import pandas as pd 
data= pd.read_excel('CHGenesOrdenadosConVecinos.xlsx',engine='openpyxl',skiprows=[0],usecols=range(8)) # Leemos el archivo de excel
data.tail() #mostramos la lectura

,Gen Abrev,Category,Name of the gene,Inicio,Final,RNASeq,Microarray,SAGE
207,PLCL2,NaN,NaN,153722829,153936148,NaN,NaN,NaN
208,TBC1D5,NaN,NaN,153936559,154524620,NaN,NaN,NaN
209,TRNAW-CCA,NaN,NaN,154244176,154244247,NaN,NaN,NaN
210,SATB1,NaN,NaN,155262593,155364610,NaN,NaN,NaN
211,KCNH8,NaN,NaN,156184812,156650711,NaN,NaN,NaN


In [13]:
Gen=data["Gen Abrev"] # Extraemos la primera columna y la guardamos en "Gen"
Gen=Gen.tolist()      # Lo tranformamos en una lista
Gen[0:5]              # Mostramos los cinco primeros datos

['TRNAS-GGA', 'LOC102173449', 'LOC102169333', 'LOC108636757', 'LOC102187712']

In [14]:
urls=[]              # Creamos una lista vacia llamada links
for name in Gen:         # Hacemos un bucle for para cada elemento de nuestra lista gen
    url = 'https://www.genecards.org/cgi-bin/carddisp.pl?gene='+name+'#expression' # Almacenamos el url para cada gen
    #print(url)
    urls.append(url) # Agregamos cada url a la lista links

In [16]:
# Importamos las librerias
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import re 
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep

link='https://www.genecards.org/cgi-bin/carddisp.pl?gene=HTR1F#expression'
# Simulación chrome
driver = webdriver.Chrome(executable_path="C:/Users/51920/Desktop/PC2-LP/PC2_LP2_UNALM/chromedriver.exe")
html=[]              # lista vacia 
for link in urls:   
    driver.get(link) # Se abre el link
   
    try:
        element = driver.find_element_by_id("expressionImage") 
        soup = BeautifulSoup(driver.page_source, features='html.parser')
        while(soup.find("area")==None):
            driver.get(link) 
            soup = BeautifulSoup(driver.page_source, features='html.parser')
        html.append(soup)
    except:
        html.append(None)
        continue
driver.quit()  # cerrar navegador

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=94.0.4606.71)
